# Reducing number of parameters of ResNet50 Model
This notebook shows how to use Low-Rank decomposition to reduce the number of parameters of a ResNet50 model. It assumes that a trained model already exist in the ```Models``` directory. Please refer to the notebook [Image Classification with ResNet50](ResNet50.ipynb) for more info about using a pretrained ResNet50 model.

## Load and evaluate the trained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus="0,1,2,3"   # Change this to match the GPUs available on your machine

testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Cafe', numWorkers=8)

model = Model.makeFromFile("Models/ResNet50.fbm", testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()

results = model.evaluate(topK=5)


Reading from "Models/ResNet50.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      9,472      
S1_L2_BN         112 112 64                             56 56 64      ReLU     MP(KSP):3 2 1    256        
S2_L1_RES2       56 56 64      2 Paths, 8 layers        56 56 256     ReLU                      76,928     
S2_L2_RES1       56 56 256     2 Paths, 6 layers        56 56 256     ReLU                      71,552     
S2_L3_RES1       56 56 256     2 Paths, 6 layers        56 56 256     ReLU                      71,552     
S3_L1_RES2       56 56 256     2 Paths, 

## Reducing number of parameters
Here we apply Low-Rank Decomposition to different layers of the model to reduce the number of parameters. We first create a list of layers we want to apply Low-Rank Decomposition, with the MSE value for each layer. We then pass this information to the [createLrModel](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.createLrModel) method to create a new fireball model saved to the file ```Models/ResNet50R.fbm```.

In [2]:
import time

# Here we use MSE=0.000055 for layers "S3_L1_RES2" through "S5_L3_RES1" and 
# use the fixed rank 400 for the last fully connected layer "S6_L1_FC":    
layerParams = [('S3_L1_RES2', 0.000055), ('S3_L2_RES1', 0.000055), ('S3_L3_RES1', 0.000055),
               ('S3_L4_RES1', 0.000055), ('S4_L1_RES2', 0.000055), ('S4_L2_RES1', 0.000055),
               ('S4_L3_RES1', 0.000055), ('S4_L4_RES1', 0.000055), ('S4_L5_RES1', 0.000055),
               ('S4_L6_RES1', 0.000055), ('S5_L1_RES2', 0.000055), ('S5_L2_RES1', 0.000055), 
               ('S5_L3_RES1', 0.000055), ('S6_L1_FC', 400)]

print('Now reducing number of network parameters ... ')
t0 = time.time()
model.createLrModel("Models/ResNet50R.fbm", layerParams)
print('Done. (%.2f Seconds)'%(time.time()-t0))

Now reducing number of network parameters ... 
  S3_L1_RES2
    Ba1 => LR(64), MSE=0.000047, Shape: (256, 128), Params: 32768->24576 (Reduction: 25.0%)
    Ba3 => LR(72), MSE=0.000049, Shape: (1152, 128), Params: 147456->92160 (Reduction: 37.5%)
    Ba5 => LR(80), MSE=0.000052, Shape: (128, 512), Params: 65536->51200 (Reduction: 21.9%)
    Bb1 => LR(88), MSE=0.000057, Shape: (256, 512), Params: 131072->67584 (Reduction: 48.4%)
  S3_L2_RES1
    Ba1 => LR(64), MSE=0.000054, Shape: (512, 128), Params: 65536->40960 (Reduction: 37.5%)
    Ba3 => LR(80), MSE=0.000048, Shape: (1152, 128), Params: 147456->102400 (Reduction: 30.6%)
    Ba5 => LR(72), MSE=0.000060, Shape: (128, 512), Params: 65536->46080 (Reduction: 29.7%)
  S3_L3_RES1
    Ba1 => LR(40), MSE=0.000058, Shape: (512, 128), Params: 65536->25600 (Reduction: 60.9%)
    Ba3 => LR(48), MSE=0.000048, Shape: (1152, 128), Params: 147456->61440 (Reduction: 58.3%)
    Ba5 => LR(48), MSE=0.000053, Shape: (128, 512), Params: 65536->30720 (Redu

Compare the new number of parameters with the original 25,636,712. 

## Evaluating the new model
Let's see the impact of this reduction to the performance of the model.

In [3]:
model = Model.makeFromFile("Models/ResNet50R.fbm", testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()
results = model.evaluate(topK=5)


Reading from "Models/ResNet50R.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      9,472      
S1_L2_BN         112 112 64                             56 56 64      ReLU     MP(KSP):3 2 1    256        
S2_L1_RES2       56 56 64      2 Paths, 8 layers        56 56 256     ReLU                      76,928     
S2_L2_RES1       56 56 256     2 Paths, 6 layers        56 56 256     ReLU                      71,552     
S2_L3_RES1       56 56 256     2 Paths, 6 layers        56 56 256     ReLU                      71,552     
S3_L1_RES2c1     56 56 256     2 Paths,

## Re-train and evaluate
The following cell creates a "tune" dataset by sampling from the training dataset and uses it to re-train the  model for 10 epochs. For better results we could also use the whole training dataset for re-training.

If the trained model ```ResNet50RR.fbm``` is already available in the ```Models``` directory, this cell shows the results of last training. If you want to force it to do the training again, you can un-remark the lines at the beginning of the cell to delete the existing file. Note that the re-training can take up to 1 hour on a 4-GPU machine (Using the whole training dataset it could take up to 10 hours).

In [4]:
# Un-remark the following line if you want to delete the existing re-trained model and re-train it again.
# import os
# if os.path.exists( "Models/ResNet50RR.fbm" ): os.remove( "Models/ResNet50RR.fbm" )
tuneDs = ImageNetDSet.makeDatasets('tune', batchSize=256, preProcessing='Crop256Cafe', numWorkers=8)
print(tuneDs)

model = Model.makeFromFile("Models/ResNet50R.fbm", trainDs=tuneDs, testDs=testDs,
                           numEpochs=10,
                           learningRate=(0.00005, 0.000005),
                           dropOutKeep = .9,
                           optimizer="Adam",
                           saveModelFileName="Models/ResNet50RR.fbm",  # Save the re-training state ...
                           savePeriod=1, saveBest=False,               # ... every epoch
                           gpus=gpus)
model.printNetConfig()
model.initSession()
model.train()
results = model.evaluate(topK=5)

ImageNetDSet Dataset Info:
    Dataset Name ................................... tune
    Dataset Location ............................... /home/shahab/data/ImageNet/
    Number of Classes .............................. 1000
    Number of Samples .............................. 64000
    Sample Shape ................................... (224, 224, 3)
    Preprocessing .................................. Crop256Cafe
    Number of Workers .............................. 8


Reading from "Models/ResNet50R.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Network configuration:
  Input:                     Color images of size 224x224
  Output:                    Probability distributions for 1000 classes.
  Network Layers:            21
  Tower Devices:             GPU0, GPU1, GPU2, GPU3
  Total Network Parameters:  11,680,232
  Total Parameter Tensors:   363
  Trainable Tensors:         257
  Training Samples:          64,000
  Test Samples:              50,000
  Num Epochs:   

## Where do I go from here?

[Pruning ResNet50 Model](ResNet50-Prune.ipynb)

[Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)

[Exporting ResNet50 Model to ONNX](ResNet50-ONNX.ipynb)

[Exporting ResNet50 Model to TensorFlow](ResNet50-TF.ipynb)

[Exporting ResNet50 Model to CoreML](ResNet50-CoreML.ipynb)

________________

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with ResNet50](ResNet50.ipynb)
